In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import nibabel as nib
import torch
from skimage.transform import resize
from skimage.color import gray2rgb
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-04-20 18:15:22.333335: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-20 18:15:23.338558: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-20 18:15:25.130830: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
train_on_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# Load the pre-trained ResNet50 model
base_model = ResNet50(include_top=False, weights='imagenet',classes=2,input_shape=(224, 224, 3))

x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output1 = layers.Dense(1, activation='sigmoid', name='output1')(x)
output2 = layers.Dense(1, activation='sigmoid', name='output2')(x)
model = models.Model(inputs=base_model.input, outputs=[output1, output2])

2023-04-20 18:15:28.432976: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
from keras.optimizers import Adam
if train_on_gpu:
    with tf.device('/GPU:0'):  # specify GPU to use
        model = model
else:
    model = model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
labels_df =pd.read_csv('/home/sandyap/DM project/balanced_labels.csv')
with tf.device('/GPU:0'):  # specify GPU to use
        labels_df=labels_df

In [6]:
labels_df.replace({'Yes': 1, 'No': 0}, inplace=True)

In [7]:
train_labels_df, test_labels_df = train_test_split(labels_df, test_size=0.4, random_state=42)
with tf.device('/GPU:0'):  # specify GPU to use
        train_labels_df=train_labels_df
with tf.device('/GPU:0'):  # specify GPU to use
        test_labels_df=test_labels_df

In [8]:
# Loop through each image file in the folder
train_images = []
train_labels = []
k=0
for i in range(len(train_labels_df)):
    filename = train_labels_df.iloc[i]['Filename']
    k=k+1
    print(k)
    print(filename)
    facial_label = train_labels_df.iloc[i]['Recognizable-Facial-Feature']
    #print('----------------------actual------------------------')
    #print(facial_label)
    brain_label = train_labels_df.iloc[i]['Brain-Feature-Loss']
    #print(brain_label)
    
    # Load the image data
    img = nib.load('/home/sandyap/DM project/files/'+filename+'.gz')
    img_data = img.get_fdata()
    with tf.device('/GPU:0'):  # specify GPU to use
        img_data=img_data
    print('image shape')
    print(img_data.shape)
    img_data = gray2rgb(img_data)
    print(img_data.shape)
    
    # Preprocess the image data
    #img_data = np.expand_dims(img_data, axis=0)
    img_data = resize(img_data[0], (224, 224,3), anti_aliasing=True, anti_aliasing_sigma=0.2)
    print('after resizing')
    print(img_data.shape)
    img_data = preprocess_input(img_data)
    print('after using preprocess_input')
    print(img_data.shape)
    train_images.append(img_data)
    train_labels.append([facial_label, brain_label])
train_images = np.array(train_images)
train_labels = np.array(train_labels)

1
IXI576-Guys-1077-T1_bet_13.nii
image shape
(256, 256, 150)
(256, 256, 150, 3)
after resizing
(224, 224, 3)
after using preprocess_input
(224, 224, 3)
2
IXI460-Guys-0999-T1_bse_less_s43_r1.nii
image shape
(256, 256, 150)


/local_scratch/pbs.529801.pbs02/ipykernel_913358/1801713962.py:28: UserWarning: Anti-aliasing standard deviation greater than zero but not down-sampling along all axes
  img_data = resize(img_data[0], (224, 224,3), anti_aliasing=True, anti_aliasing_sigma=0.2)


(256, 256, 150, 3)
after resizing
(224, 224, 3)
after using preprocess_input
(224, 224, 3)
3
IXI081-Guys-0855-T1_bse_less_s55_r1.nii
image shape
(256, 256, 150)
(256, 256, 150, 3)
after resizing
(224, 224, 3)
after using preprocess_input
(224, 224, 3)
4
IXI360-Guys-0912-T1_bse_less_s47_r1.nii
image shape
(256, 256, 150)
(256, 256, 150, 3)
after resizing
(224, 224, 3)
after using preprocess_input
(224, 224, 3)
5
IXI372-IOP-0971-T1_bet_5.nii
image shape
(256, 256, 146)
(256, 256, 146, 3)
after resizing
(224, 224, 3)
after using preprocess_input
(224, 224, 3)
6
IXI015-HH-1258-T1_bse_default.nii
image shape
(256, 256, 150)
(256, 256, 150, 3)
after resizing
(224, 224, 3)
after using preprocess_input
(224, 224, 3)
7
IXI183-Guys-0791-T1_bse_less_s45_r1.nii
image shape
(256, 256, 150)
(256, 256, 150, 3)
after resizing
(224, 224, 3)
after using preprocess_input
(224, 224, 3)
8
IXI048-HH-1326-T1_bet_18.nii
image shape
(256, 256, 150)
(256, 256, 150, 3)
after resizing
(224, 224, 3)
after using pr

In [9]:
print(train_images.shape)

(670, 224, 224, 3)


In [10]:
#balanced labels model
history=model.fit(train_images, [train_labels[:,0], train_labels[:,1]], epochs=30, batch_size=32)

Epoch 1/30
21/21 [==============================] - 21s 781ms/step - loss: 1.3747 - output1_loss: 0.6770 - output2_loss: 0.6977 - output1_accuracy: 0.6119 - output2_accuracy: 0.5910
Epoch 2/30
21/21 [==============================] - 16s 768ms/step - loss: 1.3350 - output1_loss: 0.6688 - output2_loss: 0.6663 - output1_accuracy: 0.6075 - output2_accuracy: 0.6194
Epoch 3/30
 9/21 [===========>..................] - ETA: 9s - loss: 1.2579 - output1_loss: 0.6353 - output2_loss: 0.6226 - output1_accuracy: 0.6979 - output2_accuracy: 0.7049 

KeyboardInterrupt: 

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# create an instance of ImageDataGenerator with desired augmentation parameters
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, 
                             shear_range=0.2, zoom_range=0.2, horizontal_flip=True, vertical_flip=True)

# create a generator for training data
train_generator = datagen.flow(train_images, train_labels, batch_size=64)
print(len(train_generator))

11


In [12]:
# balanced + augmented
history = model.fit(train_generator, epochs=30, steps_per_epoch=len(train_images) // 64)

Epoch 1/30


2023-04-20 18:19:05.737236: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


10/10 [==============================] - 17s 1s/step - loss: 1.4860 - output1_loss: 0.7546 - output2_loss: 0.7314 - output1_accuracy: 0.5008 - output2_accuracy: 0.4992
Epoch 2/30
10/10 [==============================] - 14s 1s/step - loss: 1.4075 - output1_loss: 0.7029 - output2_loss: 0.7046 - output1_accuracy: 0.4992 - output2_accuracy: 0.5008
Epoch 3/30
10/10 [==============================] - 14s 1s/step - loss: 1.3980 - output1_loss: 0.6980 - output2_loss: 0.7001 - output1_accuracy: 0.5008 - output2_accuracy: 0.4992
Epoch 4/30
10/10 [==============================] - 14s 1s/step - loss: 1.3933 - output1_loss: 0.6958 - output2_loss: 0.6975 - output1_accuracy: 0.4992 - output2_accuracy: 0.5008
Epoch 5/30
10/10 [==============================] - 14s 1s/step - loss: 1.3940 - output1_loss: 0.6969 - output2_loss: 0.6971 - output1_accuracy: 0.5008 - output2_accuracy: 0.4992
Epoch 6/30
10/10 [==============================] - 14s 1s/step - loss: 1.3894 - output1_loss: 0.6951 - output2_loss

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# Get the training and validation loss values from the history object
op1_loss = history.history['output1_loss']
op2_loss = history.history['output2_loss']
op1_acc=history.history['output1_accuracy']
op2_acc=history.history['output2_accuracy']

# Create a plot of the training and validation loss over each epoch
#plt.plot(train_loss, label='Training Loss')
plt.plot(op1_acc, label='output1 accuracy')
plt.plot(op2_acc, label='output2 accuracy')
plt.xlabel('Epoch')
plt.ylabel('accuracy')
plt.title('Epoch vs accuracy')
plt.legend()
plt.show()
plt.plot(op1_loss, label='output1 loss')
plt.plot(op2_loss, label='output2 loss')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.title('Epoch vs loss')
plt.legend()
plt.show()


In [ ]:
# Loop through each image file in the folder
test_images = []
test_labels = []
k=1
for i in range(len(test_labels_df)):
    filename = test_labels_df.iloc[i]['Filename']
    k=k+1
    print(i)
    print(filename)
    facial_label = test_labels_df.iloc[i]['Recognizable-Facial-Feature']
    print('----------------------actual------------------------')
    print(facial_label)
    brain_label = test_labels_df.iloc[i]['Brain-Feature-Loss']
    print(brain_label)
    
    # Load the image data
    img = nib.load('/home/sandyap/DM project/files/'+filename+'.gz')
    img_data = img.get_fdata()
    with tf.device('/GPU:0'):  # specify GPU to use
        img_data=img_data
    print(img_data.shape)
    img_data = gray2rgb(img_data)
    print(img_data.shape)
    
    # Preprocess the image data
    #img_data = np.expand_dims(img_data, axis=0)
    img_data = resize(img_data[0], (224, 224,3), anti_aliasing=True, anti_aliasing_sigma=0.2)
    print('debug1')
    print(img_data.shape)
    img_data = preprocess_input(img_data)
    print('debug2')
    print(img_data.shape)
    
    '''preds = model.predict(img_data)
    print('debug3')
    pred_facial_prob = preds[0][0]
    pred_brain_prob = preds[0][1]
    print('-------------------predicted-----------------')
    print(pred_facial_prob)
    print(pred_brain_prob)
    # Check if the predicted label matches the actual label
    pred_facial_label = 0 if pred_facial_prob >= 0.5 else 1
    pred_brain_label = 0 if pred_brain_prob >= 0.5 else 1
    
    # Check if the predicted label matches the actual label
    if pred_facial_label == facial_label: #and pred_brain_label == brain_label:
        print('Facial features are properly removed in', filename)
    else:
        print('Facial features are not properly removed in', filename)
    if  pred_brain_label == brain_label:
        print('brain voxels are properly removed in', filename)
    else:
        print('brain voxels are not properly removed in', filename)'''
    test_images.append(img_data)
    test_labels.append([facial_label, brain_label])
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
loss, test_loss1,test_loss2, test_acc1, test_acc2 = model.evaluate(test_images, [test_labels[:,0], test_labels[:,1]])
print('Test Accuracy 1:', test_acc1)
print('Test Accuracy 2:', test_acc2)
